In [1]:
import numpy as np
import torch
import sys
import os
import time
import xlwt
import csv
import random
import math
import copy
import pickle
import gc

# masterPath = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.insert(1, os.path.join(masterPath, 'centralRepo'))
sys.path.insert(1, '../centralRepo')
from eegDataset import eegDataset
from baseModel import baseModel
from networks import *
import transforms
from saveData import fetchData
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [2]:
config = {}
config['modelArguments'] = {'nChan': 22, 'nTime': 1000, 'dropoutP': 0.5,
                                    'nBands':9, 'm' : 32, 'temporalLayer': 'LogVarLayer',
                                    'nClass': 4, 'doWeightNorm': True}

# Training related details    
config['modelTrainArguments'] = {'stopCondi':  {'c': {'Or': {'c1': {'MaxEpoch': {'maxEpochs': 1000, 'varName' : 'epoch'}},
                                                   'c2': {'NoDecrease': {'numEpochs' : 200, 'varName': 'valInacc'}} } }},
      'classes': [0,1], 'sampler' : 'RandomSampler', 'loadBestModel': True,
      'bestVarToCheck': 'valInacc', 'continueAfterEarlystop':False,'lr': 1e-3}

config['batchSize'] = 16
nGPU = 0

In [3]:
config['inDataPath'] = '../../data/bci42a/multiviewPython'
config['inLabelPath'] = '../../data//bci42a/multiviewPython/dataLabels.csv'
fbcnet_dataset = eegDataset(dataPath = config['inDataPath'], dataLabelsPath= config['inLabelPath'], preloadData = False, transform= None)
FBCNET_ROOT_PATH = '../../output/bci42a/ses2Test/fbcnet_cross_subs/'
fbcnet = FBCNet(**config['modelArguments'])

config['inDataPath'] = '../../data/bci42a/rawPython'
config['inLabelPath'] = '../../data//bci42a/rawPython/dataLabels.csv'
eegnet_dataset = eegDataset(dataPath = config['inDataPath'], dataLabelsPath= config['inLabelPath'], preloadData = False, transform= None)
EEGNET_ROOT_PATH = '../../output/bci42a/ses2Test/eegnet_cross_subject/'
eegnet = eegNet(**config['modelArguments'])

models = [('fbcnet', fbcnet, fbcnet_dataset, FBCNET_ROOT_PATH), ('eegnet', eegnet, eegnet_dataset, EEGNET_ROOT_PATH)]

In [4]:
def test_model(test_sub, data, model, session='1'):
        subIdxTest = [i for i, x in enumerate(data.labels) if x[3] in test_sub and x[4] == session]
        testData = copy.deepcopy(data)
        testData.createPartialDataset(subIdxTest, loadNonLoadedData = True)

        testResults, pred, act, pred_prob = model.test_preds(testData)

        del testData
        gc.collect()

        return testResults, pred, act, pred_prob

In [5]:
subs = list(range(9))
accs = []
all_preds = {}
all_pred_probs = {}
all_losses = {}
all_test_results = {}
all_acts = {}
all_train_preds = {}
all_train_pred_probs = {}
all_train_losses = {}
all_train_results = {}
for test_sub in subs:
    test_sub_str = '00' + str(test_sub + 1)
    preds = []
    pred_probs = []
    train_preds = []
    train_pred_probs = []
    results = {}
    train_results = {}
    train_losses = []
    print("Testing subject " + test_sub_str + " with all models...")
    for sub in subs:
        if(sub == test_sub):
            continue
        model_train_results = {}
        model_test_results = {}
        for model_name, net, data, ROOT_PATH in models:
            state_dict_path = os.path.join(ROOT_PATH, 'sub' + str(sub), 'finetuned_sub' + str(test_sub), 'network_state_dict.pth')
            netInitState = torch.load(state_dict_path)
            net.load_state_dict(netInitState, strict=False)
            outPathSub = None
            model = baseModel(net=net, resultsSavePath=outPathSub, batchSize= config['batchSize'], nGPU = nGPU)
            testResults, pred, act, pred_prob = test_model(test_sub_str, data, model, session='0')
            train_preds.append(pred)
            train_pred_probs.append(pred_prob)
            model_train_results[model_name] = testResults
            train_losses.append(testResults['loss'])

            testResults, pred, act, pred_prob = test_model(test_sub_str, data, model)
            preds.append(pred)
            pred_probs.append(pred_prob)
            model_test_results[model_name] = testResults
            print("Results for model", model_name, sub, testResults)
        
        train_results[sub] = model_train_results
        results[sub] = model_test_results
    
    correct = 0
    for i in range(len(act)):
        for pred in preds:
            if(pred[i] == act[i]):
                correct += 1
                break
    
    all_preds[test_sub] = preds
    all_pred_probs[test_sub] = pred_probs
    all_acts[test_sub] = act
    all_test_results[test_sub] = results
    
    all_train_preds[test_sub] = train_preds
    all_train_pred_probs[test_sub] = train_pred_probs
    all_train_losses[test_sub] = train_losses
    all_train_results[test_sub] = train_results
    
    acc = correct/len(act)
    accs.append(acc)
    print("Theoretical max accuracy:", acc)

Testing subject 001 with all models...
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model fbcnet 1 {'acc': 0.7048611111111112, 'cm': array([[51,  9, 11,  1],
       [21, 45,  6,  0],
       [ 1,  0, 54, 17],
       [ 0,  0, 19, 53]]), 'loss': 0.6606802940368652}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model eegnet 1 {'acc': 0.5034722222222222, 'cm': array([[24, 21, 15, 12],
       [17, 41,  7,  7],
       [14, 10, 40,  8],
       [ 5, 20,  7, 40]]), 'loss': 1.4141811794704862}
Code will be running on device  cuda:0


...

../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model fbcnet 7 {'acc': 0.7430555555555556, 'cm': array([[63,  3,  6,  0],
       [10, 33, 22,  7],
       [ 4,  0, 66,  2],
       [ 1,  6, 13, 52]]), 'loss': 0.6958628760443794}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model eegnet 7 {'acc': 0.6597222222222222, 'cm': array([[51, 10,  5,  6],
       [ 4, 41, 13, 14],
       [11, 10, 36, 15],
       [ 3,  3,  4, 62]]), 'loss': 1.2813310623168945}
Theoretical max accuracy: 1.0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


In [6]:
accs

[0.9965277777777778,
 0.9652777777777778,
 1.0,
 0.9930555555555556,
 0.9930555555555556,
 0.9791666666666666,
 0.9965277777777778,
 0.9930555555555556,
 1.0]

In [7]:
avg_acc = sum(accs) / len(accs)
print("Average theoretical max accuracy:", avg_acc)

Average theoretical max accuracy: 0.9907407407407407


In [8]:
accs.append(avg_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in accs])
print(tabstr)

99.653 & 96.528 & 100.000 & 99.306 & 99.306 & 97.917 & 99.653 & 99.306 & 100.000 & 99.074


In [9]:
# sub_spec_accs = [all_test_results[i][i]['acc'] for i in range(9)]
# print(sub_spec_accs)
# print("Subject specific average accuracy:", sum(sub_spec_accs) / len(sub_spec_accs))
# print("Subject specific median accuracy:", statistics.median(sub_spec_accs))

In [10]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for pred in preds:
            votes[pred[i]]+=1
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.8298611111111112
Voted accuracy: 0.625
Voted accuracy: 0.90625
Voted accuracy: 0.7256944444444444
Voted accuracy: 0.7916666666666666
Voted accuracy: 0.6145833333333334
Voted accuracy: 0.7847222222222222
Voted accuracy: 0.8402777777777778
Voted accuracy: 0.7534722222222222
Average voted accuracy: 0.7635030864197531
Median voted accuracy: 0.7847222222222222
82.986 & 62.500 & 90.625 & 72.569 & 79.167 & 61.458 & 78.472 & 84.028 & 75.347 & 76.350


### Soft probability combine

In [11]:
comb_accs = []
for test_sub in subs:
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs.shape)
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.8402777777777778
Soft probability combined accuracy: 0.6840277777777778
Soft probability combined accuracy: 0.8993055555555556
Soft probability combined accuracy: 0.7361111111111112
Soft probability combined accuracy: 0.7951388888888888
Soft probability combined accuracy: 0.6284722222222222
Soft probability combined accuracy: 0.8125
Soft probability combined accuracy: 0.8298611111111112
Soft probability combined accuracy: 0.7743055555555556
Average combined accuracy: 0.7777777777777778
Median combined accuracy: 0.7951388888888888
84.028 & 68.403 & 89.931 & 73.611 & 79.514 & 62.847 & 81.250 & 82.986 & 77.431 & 77.778


In [12]:
cms = {test_sub: {train_sub: [results['fbcnet']['cm'], results['eegnet']['cm']]  for train_sub, results in test_results.items()} for test_sub, test_results in all_train_results.items()}

In [13]:
# metrics = {test_sub: {train_sub: [np.array([2*cm[0][i][i]/(np.sum(cm[0][i,:]) + np.sum(cm[0][:,i])) for i in range(4)]), np.array([2*cm[1][i][i]/(np.sum(cm[1][i,:]) + np.sum(cm[1][:,i])) for i in range(4)])] for train_sub, cm in test_results.items()} for test_sub, test_results in cms.items()}
metrics = {test_sub: {train_sub: [np.array([cm[0][i][i]/np.sum(cm[0][i,:]) for i in range(4)]), np.array([cm[1][i][i]/np.sum(cm[1][i,:]) for i in range(4)])] for train_sub, cm in test_results.items()} for test_sub, test_results in cms.items()}

In [23]:
recall_sums = {test_sub: np.concatenate([np.array([np.sum(recalls[0]) for train_sub, recalls in test_results.items()]), np.array([np.sum(recalls[1]) for train_sub, recalls in test_results.items()])]) for test_sub, test_results in metrics.items()}

In [24]:
recall_sums[0]

array([4.        , 3.90277778, 3.94444444, 3.95833333, 3.91666667,
       3.97222222, 3.86111111, 4.        , 3.94444444, 4.        ,
       3.94444444, 3.86111111, 3.94444444, 3.97222222, 3.97222222,
       3.93055556])

## RSSP

In [15]:
comb_accs = []
for test_sub in subs:
    mlist = list(range(9))
    mlist.pop(test_sub)
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs.shape)
#     print(pred_probs[8][0])
    for i in range(len(pred_probs)):
#         print(pred_probs[i])
        for j in range(len(pred_probs[i])):
            pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i % 8]][i // 8])
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.8402777777777778
Soft probability combined accuracy: 0.6840277777777778
Soft probability combined accuracy: 0.8993055555555556
Soft probability combined accuracy: 0.7326388888888888
Soft probability combined accuracy: 0.7881944444444444
Soft probability combined accuracy: 0.6319444444444444
Soft probability combined accuracy: 0.8090277777777778
Soft probability combined accuracy: 0.8263888888888888
Soft probability combined accuracy: 0.78125
Average combined accuracy: 0.7770061728395061
Median combined accuracy: 0.7881944444444444
84.028 & 68.403 & 89.931 & 73.264 & 78.819 & 63.194 & 80.903 & 82.639 & 78.125 & 77.701


## RSV

In [16]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
    mlist = list(range(9))
    mlist.pop(test_sub)
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for j, pred in enumerate(preds):
            votes[pred[i]]+=metrics[test_sub][mlist[j % 8]][j // 8][pred[i]]
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.8298611111111112
Voted accuracy: 0.6354166666666666
Voted accuracy: 0.8993055555555556
Voted accuracy: 0.7361111111111112
Voted accuracy: 0.7847222222222222
Voted accuracy: 0.6111111111111112
Voted accuracy: 0.7708333333333334
Voted accuracy: 0.8368055555555556
Voted accuracy: 0.7465277777777778
Average voted accuracy: 0.7611882716049382
Median voted accuracy: 0.7708333333333334
82.986 & 63.542 & 89.931 & 73.611 & 78.472 & 61.111 & 77.083 & 83.681 & 74.653 & 76.119


In [17]:
comb_accs = []
for test_sub in subs:
    mlist = list(range(9))
    mlist.pop(test_sub)
    recalls = np.concatenate(recall_sums[test_sub])
    sorted_recall_idx = np.argsort(recalls)
    # print(sorted_recall_idx)
    # print(sorted_recall_idx[-5:])
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs.shape)
    pred_probs = np.take(pred_probs, sorted_recall_idx[-8:], 0)
#     print(pred_probs.shape)
    for i in range(len(pred_probs)):
        for j in range(len(pred_probs[i])):
            pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i % 8]][i // 8])
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Top recall scaled soft probability combined accuracy:", voted_acc)
    
avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Top recall scaled soft probability combined accuracy: 0.7986111111111112
Top recall scaled soft probability combined accuracy: 0.65625
Top recall scaled soft probability combined accuracy: 0.8923611111111112
Top recall scaled soft probability combined accuracy: 0.7395833333333334
Top recall scaled soft probability combined accuracy: 0.7395833333333334
Top recall scaled soft probability combined accuracy: 0.6076388888888888
Top recall scaled soft probability combined accuracy: 0.8368055555555556
Top recall scaled soft probability combined accuracy: 0.8055555555555556
Top recall scaled soft probability combined accuracy: 0.7708333333333334
Average combined accuracy: 0.7608024691358025
Median combined accuracy: 0.7708333333333334
79.861 & 65.625 & 89.236 & 73.958 & 73.958 & 60.764 & 83.681 & 80.556 & 77.083 & 76.080


In [27]:
for i in range(2,9):
    comb_accs = []
    print("Top-" + str(i))
    for test_sub in subs:
        mlist = list(range(9))
        mlist.pop(test_sub)
        sorted_recall_idx = np.argsort(recall_sums[test_sub])
        # print(sorted_recall_idx)
        # print(sorted_recall_idx[-5:])
        pred_probs = np.array(all_pred_probs[test_sub])
        n, h, w, c = pred_probs.shape
        pred_probs = np.reshape(pred_probs, (n, h*w, c))
    #     print(pred_probs.shape)
        pred_probs = np.take(pred_probs, sorted_recall_idx[-i:], 0)
    #     print(pred_probs.shape)
#         for i in range(len(pred_probs)):
#             for j in range(len(pred_probs[i])):
#                 pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i % 8]][i // 8])
        class_probs = np.sum(pred_probs, axis=0)
        soft_preds = np.argmax(class_probs, axis=1)
        correct = 0
        for pred, act_val in zip(soft_preds, all_acts[test_sub]):
            if(pred == act_val):
                correct += 1
        voted_acc = correct / len(all_acts[test_sub])
        comb_accs.append(voted_acc)
        print("Top recall scaled soft probability combined accuracy:", voted_acc)

    avg_comb_acc = sum(comb_accs) / len(comb_accs)
    print("Average combined accuracy:", avg_comb_acc)
    print("Median combined accuracy:", statistics.median(comb_accs))
    comb_accs.append(avg_comb_acc)
    tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
    print(tabstr)
    print()
    print()

Top-2
Top recall scaled soft probability combined accuracy: 0.6041666666666666
Top recall scaled soft probability combined accuracy: 0.5277777777777778
Top recall scaled soft probability combined accuracy: 0.7638888888888888
Top recall scaled soft probability combined accuracy: 0.6180555555555556
Top recall scaled soft probability combined accuracy: 0.6597222222222222
Top recall scaled soft probability combined accuracy: 0.5138888888888888
Top recall scaled soft probability combined accuracy: 0.7465277777777778
Top recall scaled soft probability combined accuracy: 0.5729166666666666
Top recall scaled soft probability combined accuracy: 0.6875
Average combined accuracy: 0.6327160493827161
Median combined accuracy: 0.6180555555555556
60.417 & 52.778 & 76.389 & 61.806 & 65.972 & 51.389 & 74.653 & 57.292 & 68.750 & 63.272


Top-3
Top recall scaled soft probability combined accuracy: 0.75
Top recall scaled soft probability combined accuracy: 0.6076388888888888
Top recall scaled soft probabi